In [101]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
#nltk.download()


In [102]:
Data = pd.read_csv("T.csv", sep='|' , delimiter=',' ,encoding='latin-1')

In [103]:
col = ['Material/Service Description','I06: Material Group','L0','L1','L2']

In [104]:
Train_data = Data[col]

In [105]:
Train_data.rename(columns = {'Material/Service Description' : 'Desc', 'I06: Material Group' : 'MG'}, inplace = True)

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [106]:
Train_data.head()

,Desc,MG,L0,L1,L2
0,"POWERMODULE,24V/125A,CATHELCO,8020110103",MEL010000,Electrical & Instrumentation,Electrical,Electrical Accessories
1,"PLUG,POWER, RED,16A,380-415V,RS,190-3001",MEL010000,Electrical & Instrumentation,Electrical,Electrical Accessories
2,"PLUG, ELECTRICAL, 1PH, 230V, EX",MEL010000,Electrical & Instrumentation,Electrical,Electrical Accessories
3,"PLUG, ELECTRICAL, 1PH, 230V, EX",MEL010000,Electrical & Instrumentation,Electrical,Electrical Accessories
4,"PLUG, ELECTRICAL, 1PH, 230V, EX",MEL010000,Electrical & Instrumentation,Electrical,Electrical Accessories


In [107]:
Train_data["L1"].value_counts()

Electrical         2042
E&I Bulk           1493
Instrumentation     546
Name: L1, dtype: int64

In [108]:
TrainData = Train_data.loc[Train_data.L1 == 'E&I Bulk']

In [109]:
#TrainData

In [110]:
TrainData["L1"].value_counts()
TrainData["L2"].value_counts()

Cables, Cords and Wires                438
Cable Racks, Trays and Ladders         414
Switches and Control Equipment         323
Transits and Glands                    231
Instrument Tubing and Fittings          73
Instrument and Communication Cables     14
Name: L2, dtype: int64

In [111]:
import re
def clean_str(string):
    string = re.sub(r"\n","", string)
    string = re.sub(r"\r", "", string)
    #string = re.sub(r"[0-9]","digit", string)
    string = re.sub(r"\'", " ", string)
    string = re.sub(r"\"", " ", string)
    string = re.sub(r'\(.*?\)', '', string)
    string = re.sub(r"/", " ", string)
    string = re.sub(r":", " ", string)
    string = re.sub(r"-", " ", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"\+", " ", string)
    string = re.sub(r"&"," ", string)
    string = re.sub(r"#"," ", string)
    string = re.sub(r"\."," ", string)
    string = re.sub(r"\b\w+\d+\b"," ", string)
    string = re.sub(r"\b\d+\w+\b"," ", string)
    string = re.sub(r"\b\w+\d+\w+\b"," ", string)
    string = re.sub(r"[0-9]","", string)
    string = re.sub(r"\\b[A-Za-z] \\b|\\b [A-Za-z]\\b", ' ', string)
    string = re.sub(r'\b\w\b', '', string)
    string = re.sub(r'\b\w\w\b', '', string)
    string = re.sub(r'" "', '', string)
    string = re.sub(' +', ' ', string)
    #string = re.sub('cons\s', "consumables " " ", str)
    return string.strip().lower()


In [112]:
TrainData['Feature'] = TrainData['Desc'].apply(lambda x: clean_str(x))

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [113]:
def replace_str(string):
    string = re.sub("cons\s", "consumables " " ", string)
    string = re.sub("chem\s", "chemicals " " ", string)
    string = re.sub("geotech\s", "geotechnical" " ", string)
    string = re.sub("test\s", "testing " " ", string)
    string = re.sub("inst\s", "instrument " " ", string)
    string = re.sub("transport\s", "transportation " " ", string)
    string = re.sub("rig\s", "rigging " " ", string)
    string = re.sub("lift\s", "lifting " " ", string)
    string = re.sub("cables\s", "cable  " " ",     string)
    string = re.sub("fittin\s", "fittings " " ", string)
    string = re.sub("peri\s", "peripherals " " ", string)
    string = re.sub(r'" "', '', string)
   
    return string.strip().lower()

In [114]:
TrainData['Feature'] = TrainData['Feature'].apply(replace_str)

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [115]:
#TrainData

In [116]:
#Clean = pd.DataFrame(TrainData).to_csv('Clean.csv')

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(TrainData['Feature'], TrainData['L2'],test_size=0.3, random_state = 2)
count_vect = CountVectorizer(ngram_range=(1, 2), stop_words='english')
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


Dimensionality reduction using PCA()

In [126]:
X_train_dense = X_train_tfidf.todense() 
pca = PCA()
X_train_PCA = pca.fit_transform(X_train_dense)

In [127]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train_PCA, y_train)

In [128]:
X_test = count_vect.transform(X_test)
X_test_dense = X_test.todense()
X_test_pca = pca.transform(X_dense)

In [135]:
X_train_counts.shape

(1045, 1234)

In [95]:

y_pred = clf.predict(X_pca)

In [96]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5022321428571429


In [97]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[  5 124   0   0   1   0]
 [ 26  67   0   0   1  36]
 [  3   1  16   0   1   0]
 [  1   3   0   0   0   0]
 [  6   3   2   0  84   2]
 [  0  13   0   0   0  53]]
                                     precision    recall  f1-score   support

     Cable Racks, Trays and Ladders       0.12      0.04      0.06       130
            Cables, Cords and Wires       0.32      0.52      0.39       130
     Instrument Tubing and Fittings       0.89      0.76      0.82        21
Instrument and Communication Cables       0.00      0.00      0.00         4
     Switches and Control Equipment       0.97      0.87      0.91        97
                Transits and Glands       0.58      0.80      0.68        66

                           accuracy                           0.50       448
                          macro avg       0.48      0.50      0.48       448
                       weighted avg       0.46      0.50      0.47       448



C:\Users\user\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


The accuracy is low, because we lost most information during dimensionality reduction.


For this problem, I would recommend:

1. Do not reduce dimensions mathematically. Instead, preprocess the text lingustically: drop the stop-words, stem or lemmatize the rest of words, and drop the words which occure less than k times. It will bring your dimensionality from 1234K to something like 800 without serious loss of information.


2. Use a pre-trained dimensionality reducer, like word2vec or fastText, to extract features from your text. There exist pre-trained word2vec models in the Internet, for multiple languages, and several dimensionalities (like 200, 1000, etc.).
